In [1]:
import math, random
import numpy as np

def calculate_impact_point_on_ground(x, z, yaw_deg, pitch_deg, gravity=9.81):
    initial_speed = 60

    # 각도를 라디안으로 변환
    yaw = math.radians(yaw_deg)
    pitch = math.radians(pitch_deg)

    # 3D 방향 벡터 계산
    dx = math.cos(pitch) * math.sin(yaw)
    dy = math.sin(pitch)
    dz = math.cos(pitch) * math.cos(yaw)

    # 초기 속도 벡터 성분
    v0x = initial_speed * dx
    v0y = initial_speed * dy
    v0z = initial_speed * dz

    # 수직(y) 방향으로 다시 지면(y = 0)에 도달하는 시간
    if v0y <= 0:
        return (x, 0, z)
    
    t_impact = 2 * v0y / gravity
    
    # x, z 평면 착탄 위치
    impact_x = x + v0x * t_impact
    impact_z = z + v0z * t_impact

def invert_pitch_from_impact_distance(self, d, gravity=9.81, initial_speed=60):
    factor = 733.34 # 상수 값 (앞에서 계산한 근사치)
    arg = (2 * d) / factor

    # arg 값이 [-1, 1] 범위에 있어야 함
    if abs(arg) > 1.0:
        #발사 불가능 거리, pitch=0 (평사)
        return 0.0
    pitch_rad = 0.5 * math.asin(arg)
    return math.degrees(pitch_rad)

def angle_diff(self, a, b):
    # 두 각도 사이의 최소 차이 (0~180도)
     return (a - b + 180) % 360 - 180

def script_action(self):
    # 포탑을 적 방향으로 조준하고 발사
    dx = self.enemy_x - self.tank_x
    dy = self.enemy_y - self.tank_y
    distance = math.sqrt(dx**2 + dy**2)

    target_yaw = (math.degrees(math.atan2(dx, dy))) % 360.0
    target_pitch = self.invert_pitch_from_impact_distance(distance)

    yaw_error = self.angle_diff(self.turret_x, target_yaw)
    pitch_error = target_pitch - self.turret_y

    turret_dx = random.uniform(0.1, 1.0) if yaw_error > 2.0 else (random.uniform(-1.0, -0.1) if yaw_error < -2.0 else random.uniform(-0.05, 0.05))
    turret_dy = random.uniform(0.1, 1.0) if pitch_error > 2.0 else (random.uniform(-1.0, -0.1) if pitch_error < -2.0 else random.uniform(-0.05, 0.05))
    
    fire = 1.0 if abs(yaw_error) < 2.0 and abs(pitch_error) < 2.0 and self.cooldown_norm >= 0.99 else 0.0
    return np.array([turret_dx, turret_dy, fire], dtype=np.float32)